In [8]:
import boto3
import os
from botocore.exceptions import ClientError

# --- Configuration (UPDATE THESE VALUES) ---
# NOTE: S3 object keys are case-sensitive!
AWS_REGION = 'us-east-1' # <-- REPLACE with your bucket's actual AWS region
BUCKET_NAME = 'qa-agent-files-updated' # <-- !! REPLACE with your S3 bucket name
S3_OBJECT_KEY = 'recipes.csv' # <-- !! REPLACE with the full path/key of the file


# Define the target directory path within your home directory
TARGET_DIR = '/home/sagemaker-user/codespace/nlp_assignments/QA_agent2/data'

# Define the full local path for the downloaded file
LOCAL_FILE_PATH = os.path.join(TARGET_DIR, S3_OBJECT_KEY.split('/')[-1])
# Using S3_OBJECT_KEY.split('/')[-1] ensures the local filename is just 'recipes.csv', 
# even if the key includes folders like 'raw-data/recipes.csv'.

# --- Download Logic ---

# Initialize S3 client with the explicit region
print(f"Connecting to S3 in region: {AWS_REGION}...")
s3_client = boto3.client('s3', region_name=AWS_REGION)

try:
    # 1. Ensure the target directory exists before download
    if not os.path.exists(TARGET_DIR):
        os.makedirs(TARGET_DIR, exist_ok=True) 
        print(f"Created local directory: {TARGET_DIR}")

    # 2. Log the target S3 path for verification
    print(f"Attempting to download s3://{BUCKET_NAME}/{S3_OBJECT_KEY}")
    print(f"Saving locally to: {LOCAL_FILE_PATH}")

    # 3. Use the download_file method
    s3_client.download_file(BUCKET_NAME, S3_OBJECT_KEY, LOCAL_FILE_PATH)
    
    print("\n✅ Download Successful!")
    print(f"Successfully downloaded '{S3_OBJECT_KEY}' to '{LOCAL_FILE_PATH}'")

# Handle the specific S3 ClientError (404 Not Found, 403 Forbidden, etc.)
except ClientError as e:
    # 'Error Code' is typically what you check for the 404 (NoSuchKey)
    error_code = e.response['Error']['Code']
    
    if error_code == '404' or error_code == 'NoSuchKey':
        print("\n❌ S3 Error: File Not Found (404/NoSuchKey)")
        print("Please check that BUCKET_NAME and S3_OBJECT_KEY are spelled EXACTLY right (S3 keys are case-sensitive!).")
        print(f"Configured Path: s3://{BUCKET_NAME}/{S3_OBJECT_KEY}")
    elif error_code == '403' or error_code == 'AccessDenied':
        print("\n❌ S3 Error: Access Denied (403)")
        print("Your IAM Role (SageMaker execution role) likely lacks s3:GetObject permission for this file.")
    else:
        print(f"\n❌ An unexpected S3 ClientError occurred: {error_code}")
        print(e)

except PermissionError:
    print(f"\n❌ Local System Error: Permission denied to write to {LOCAL_FILE_PATH}")
    print("Please ensure the 'sagemaker-user' has write permissions to that directory.")

except Exception as e:
    print(f"\n❌ A general error occurred: {e}")

Connecting to S3 in region: us-east-1...
Attempting to download s3://qa-agent-files-updated/recipes.csv
Saving locally to: /home/sagemaker-user/codespace/nlp_assignments/QA_agent2/data/recipes.csv



✅ Download Successful!
Successfully downloaded 'recipes.csv' to '/home/sagemaker-user/codespace/nlp_assignments/QA_agent2/data/recipes.csv'
